In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')
import numpy as np
import itertools
import matplotlib.pyplot as plt
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/afareen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/afareen/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
def get_all_data():
    dataTrue = pd.read_csv('True.csv')
    dataTrue['Label'] = 0 #not fake news
    dataFalse = pd.read_csv('Fake.csv')
    dataFalse['Label'] = 1 #fake news
    list = [dataTrue,dataFalse]
    data = pd.concat(list).reset_index(drop=True)
    return data

In [3]:
#get rid of uppercase, new lines, non-alphabetical characters
def preprocess_data(data):
    for i in range(len(data)):
        if i%100 == 0: 
            print(i)
        title_temp = data['title'][i].lower()
        title_temp = title_temp.replace('\n',' ')
        title_temp = ''.join(c for c in title_temp if c.isalpha() | (c == ' '))
        #print(title_temp)
        text_temp = data['text'][i].lower()
        text_temp = text_temp.replace('\n',' ')
        text_temp = ''.join(c for c in text_temp if c.isalpha() | (c == ' '))
        tokens_title = word_tokenize(title_temp)
        tokens_temp = word_tokenize(text_temp)
        tokens = tokens_title + tokens_temp 
#         no_stopwords = [word for word in tokens if not word in stopwords.words()]
#         data['text'][i] = no_stopwords
        data['text'][i] = tokens
        #print(data['text'][i])
    return data 

In [ ]:
fullDataset = get_all_data()
fixedDataset = preprocess_data(fullDataset)
fixedDataset.head()

0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
188

,title,text,subject,date,Label
0,"As U.S. budget fight looms, Republicans flip t...","[as, us, budget, fight, looms, republicans, fl...",politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,"[us, military, to, accept, transgender, recrui...",politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,"[senior, us, republican, senator, let, mr, mue...",politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,"[fbi, russia, probe, helped, by, australian, d...",politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,"[trump, wants, postal, service, to, charge, mu...",politicsNews,"December 29, 2017",0


In [ ]:
fixedDataset.to_csv('Cleaned-Data.csv')

In [ ]:
relevant_data = fixedDataset[['text','Label']]
relevant_data.head()

,text,Label
0,"[as, us, budget, fight, looms, republicans, fl...",0
1,"[us, military, to, accept, transgender, recrui...",0
2,"[senior, us, republican, senator, let, mr, mue...",0
3,"[fbi, russia, probe, helped, by, australian, d...",0
4,"[trump, wants, postal, service, to, charge, mu...",0


In [ ]:
train_indices = np.random.rand(len(relevant_data)) < 0.8
train = relevant_data[train_indices].reset_index(drop=True)
test = relevant_data[~train_indices].reset_index(drop=True)
print(train.head())
print(len(train))
print(test.head())
print(len(test))

                                                text  Label
0  [as, us, budget, fight, looms, republicans, fl...      0
1  [us, military, to, accept, transgender, recrui...      0
2  [senior, us, republican, senator, let, mr, mue...      0
3  [trump, wants, postal, service, to, charge, mu...      0
4  [white, house, congress, prepare, for, talks, ...      0
35870
                                                text  Label
0  [fbi, russia, probe, helped, by, australian, d...      0
1  [trump, says, russia, probe, will, be, fair, b...      0
2  [factbox, trump, on, twitter, dec, approval, r...      0
3  [alabama, official, to, certify, senatorelect,...      0
4  [new, york, governor, questions, the, constitu...      0
9028


In [ ]:
#train model
def training_step(training_data, vectorizer):
    training_text = []
    training_result = []
    for i in range(len(training_data)):
        #training_text.append(training_data['text'][i])
        training_text.append(' '.join(str(x) for x in training_data['text'][i]))
        training_result.append(training_data['Label'][i])
    training_text = vectorizer.fit_transform(training_text)   
    classifier = MultinomialNB().fit(training_text, training_result) #remove toarray for bernoulli/multi
#     print("Classifier: " + str(classifier))
#     print("Classifier log prior: " + str(classifier.class_log_prior_))
#     print("Feature log prob: " + str(classifier.feature_log_prob_))
    classifier.get_params()
    return classifier

In [ ]:
#return block of text and its classifier prediction
def analyze_text(classifier, vectorizer, text):
#     print("Classifier prediction: " + str(classifier.predict(vectorizer.transform([text]))))
    return text, classifier.predict(vectorizer.transform([text]))

In [ ]:
#evaluate model performance
def evaluate(evaluation_data, vectorizer):
    evaluation_text = []
    evaluation_result = []
    #separate features/labels
    for i in range(len(evaluation_data)):
        #training_text.append(training_data['text'][i])
        evaluation_text.append(' '.join(str(x) for x in evaluation_data['text'][i]))
        evaluation_result.append(evaluation_data['Label'][i])
    #check accuracy of classifier on evaluation data
    total = len(evaluation_text)
    corrects = 0
    for i in range(0, total):
        result = classifier.predict(vectorizer.transform([evaluation_text[i]])) #remove toarray
        text = evaluation_text[i]
        corrects += 1 if result[0] == evaluation_result[i] else 0
    print("Accuracy: ", corrects * 100 / total)
    return corrects * 100 / total

In [ ]:
v = CountVectorizer('binary=true')
classifier = training_step(train, v)
acc = evaluate(test, v)
print(acc)

Accuracy:  96.24501550731058
96.24501550731058
